In [2]:
#need to read in a layer
#test a probe and write the results 

#multiple layers? --> last three of hte xl embeddings 

In [17]:
import pandas as pd
df = pd.read_csv('./data/animals_true_false.csv')
source_labels = df['label']

In [18]:
import os
import numpy as np

# Set the folder path
folder_path = "data/embeddings/gpt2/xl"

merged_embeddings = []
basic_embeddings = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is a numpy file
    if filename.endswith(".npy"):
        # Load the embedding data from the file
        embedding_data = np.load(file_path)
        
        # Check if the filename contains "merged"
        if "merged" in filename:
            merged_embeddings.append(embedding_data)
        else:
            basic_embeddings.append(embedding_data)

In [19]:
from itertools import combinations

def generate_combinations(x, y):
    """
    Generates all combinations of numbers from x to y.
    
    Args:
        x (int): The lower bound of the range (inclusive).
        y (int): The upper bound of the range (inclusive).
        
    Returns:
        list: A list of tuples, where each tuple represents a combination
        of numbers from x to y.
    """
    nums = list(range(x, y+1))  # Generate a list of numbers from x to y
    combinations_list = []
    
    # Generate combinations of different lengths
    for length in range(1, len(nums)+1):
        combs = combinations(nums, length)
        combinations_list.extend(combs)
    
    return combinations_list

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def probe(embeddings, labels, layer_list, input_dim=1600):
  X = np.array(embeddings)  
  y = np.array(labels)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Define and compile model
  model = Sequential()
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, y_test)

  layer_str = '-'.join(map(str, layer_list))
  save_data(test_loss, test_acc, layer_str)

def save_data(test_loss, test_acc, layer):
  write_data = f'{layer}: {test_loss} {test_acc}'
  with open('./statistics/Layer_Data/gpt2_xl_location_multilayer_3_merged.txt', 'a') as file:
    file.write(f'{write_data}\n')

In [21]:
indices = generate_combinations(45,48)

In [22]:
def parse_embeddings(embeddings_list, index_list):
    flat_list = []
    for index in index_list:
        flat_list.extend(embeddings_list[index])
    return flat_list  

In [23]:
for index_list in indices:
  #construct embeddings for this list 
  #create the label amount by multiplying 
  embeddings = parse_embeddings(merged_embeddings, index_list)
  labels = df['label'] * len(index_list)
  probe(embeddings=embedding_data, labels=labels, layer_list=index_list)

Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4750 - loss: 43.2882 - val_accuracy: 0.4691 - val_loss: 2.2632
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5064 - loss: 13.9154 - val_accuracy: 0.4691 - val_loss: 0.7479
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4917 - loss: 0.8161 - val_accuracy: 0.5062 - val_loss: 0.6915
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4982 - loss: 0.7131 - val_accuracy: 0.4938 - val_loss: 0.6938
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5032 - loss: 0.7220 - val_accuracy: 0.4815 - val_loss: 0.6953
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5112 - loss: 0.6917 - val_accuracy: 0.4815 - val_loss: 0.6973
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5558 - loss: 0.6987 - val_accuracy: 0.4938 - val_loss: 0.7024
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4968 - loss: 0.7072 - val_accuracy: 0.4753 - val_los